In [1]:
import torch

# 1️⃣ Decide once, at start-up, where to run everything
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Running Kokoro on {DEVICE}")


Running Kokoro on cpu


In [2]:
import PyPDF2
from kokoro import KPipeline
import soundfile as sf
import numpy as np
from tqdm.notebook import tqdm  # <- for the progress bar
import re


# 1. Load PDF and extract text
abook_file = "endymion_50-54"
pdf_path = f'/home/alex/Projects/TTS/kokoro/Dan Simmons - Endymion/{abook_file}.pdf'
with open(pdf_path, "rb") as fh:
    pdf_reader = PyPDF2.PdfReader(fh)
    text = "\n".join(page.extract_text() or "" for page in pdf_reader.pages)

# 2. Initialize Kokoro TTS pipeline
pipeline = KPipeline(lang_code='b', device=DEVICE) 

# 3. Count how many chunks will be generated for more accurate progress (optional!)
split_pat = re.compile(r'[.”]\s*\n')          # keep the delimiter simpler
chunks = [c.strip() for c in split_pat.split(text) if c.strip()]
num_chunks = len(chunks)

# 4. Generate audio from PDF text
generator = pipeline(
    text,
    voice="bf_emma",
    speed=0.8,
    split_pattern=split_pat.pattern     # pass the raw pattern
)
all_audio = []
for i, (*_, audio) in tqdm(
        enumerate(generator, 1),
        total=num_chunks,
        desc="Synthesizing",
        unit="chunk"):
    all_audio.append(audio)

if all_audio:
    merged_audio = np.concatenate(all_audio, axis=0)
    sf.write(
    file=pdf_path.split('.')[0] + '.mp3',
    data=merged_audio, 
    samplerate=24000, 
    format='MP3',
    bitrate_mode='CONSTANT', 
    compression_level=.5
    )
    print(f"Saved to {abook_file}.mp3")
else:
    print("No audio generated.")

/home/alex/Projects/TTS/.venv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/alex/Projects/TTS/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Synthesizing:   0%|          | 0/352 [00:00<?, ?chunk/s]

Saved to endymion_50-54.mp3
